In [6]:
import os
import tensorflow as tf
import numpy as np
from PIL import Image, ImageOps # Necesitas: pip install pillow  # Cambia esto por la ruta de tu imagen

print("cwd:", os.getcwd())
print("files (top-level):", os.listdir('.'))

# 1) permite pasar ruta explícita por variable de entorno
model_path = os.getenv('MODEL_PATH')  # opcional: export MODEL_PATH=/ruta/mi_modelo.keras

# 2) búsqueda recursiva si no hay ruta explícita
def find_models(root='.'):
    exts = ('.keras', '.h5')
    for dirpath, _, files in os.walk(root):
        for f in files:
            if f.endswith(exts):
                yield os.path.join(dirpath, f)

def is_ignored_path(p):
    ignore_tokens = (os.sep + '.venv' + os.sep, 'site-packages', os.sep + 'tests' + os.sep)
    return any(tok in p for tok in ignore_tokens)

if model_path and os.path.exists(model_path):
    print("Cargando modelo desde MODEL_PATH:", model_path)
    model = tf.keras.models.load_model(model_path, compile=False)
else:
    candidates = list(find_models('.'))
    # Prioriza archivos en el directorio actual y evita rutas de entorno/paquetes
    candidates = sorted(candidates, key=lambda p: (is_ignored_path(p), os.path.dirname(p) != '.', p))
    if candidates:
        print("Modelos encontrados (ordenados):", candidates)
        for candidate in candidates:
            try:
                print("Intentando cargar:", candidate)
                model = tf.keras.models.load_model(candidate, compile=False)
                print("Modelo cargado desde:", candidate)
                break
            except Exception as e:
                print("No es un modelo Keras válido:", candidate, "->", e)
        else:
            raise FileNotFoundError(
                "Se encontraron archivos .h5/.keras pero ninguno contiene un modelo Keras válido. "
                "Coloca 'mi_modelo.keras' en la carpeta del notebook o exporta MODEL_PATH."
            )
    else:
        raise FileNotFoundError(
            "Model file not found. Coloca 'mi_modelo.keras' en la carpeta del notebook, "
            "o exporta la ruta: export MODEL_PATH=/ruta/mi_modelo.keras. "
            "Para guardar: model.save('mi_modelo.keras')"
        )

def predecir_ropa(ruta_imagen):
        # 2. CARGAR Y PRE-PROCESAR LA IMAGEN (La parte crítica)
        # A. Abrir imagen como escala de grises ('L')
        img = Image.open(ruta_imagen).convert('L')

        # B. INVERTIR COLORES
        # Si el fondo es blanco, lo volvemos negro para que coincida con el entrenamiento
        img = ImageOps.invert(img)

        # C. REDIMENSIONAR
        # El modelo es miope, solo ve 28x28 píxeles.
        img = img.resize((28, 28))

        # D. CONVERTIR A NÚMEROS
        # De imagen a matriz NumPy y normalizar (0 a 1)
        img_array = np.array(img) / 255.0
        
        # E. DAR FORMA DE LOTE (BATCH)
        # El modelo espera una lista de fotos, no una sola.
        # Transformamos (28, 28) -> (1, 28, 28, 1)
        img_array = img_array.reshape(1, 28, 28, 1)

        # 3. PREDICCIÓN
        prediction = model.predict(img_array)

        # 4. INTERPRETAR RESULTADO
        clases = ['Camiseta', 'Pantalón', 'Jersey', 'Vestido', 'Abrigo',
                'Sandalia', 'Camisa', 'Zapatilla', 'Bolso', 'Bota']
        
        indice_ganador = np.argmax(prediction) # ¿Cuál neurona gritó más fuerte?
        confianza = np.max(prediction)         # ¿Qué tan fuerte gritó? (0-1)

        print(f"------------------------------------------------")
        print(f"Imagen: {ruta_imagen}")
        print(f"Predicción: {clases[indice_ganador].upper()}")
        print(f"Confianza: {confianza*100:.2f}%")
        print(f"------------------------------------------------")

cwd: c:\Users\Miguel\Desktop\Miguelon\GH\Machine_Learning_Projects
files (top-level): ['.git', '.venv', 'Artificial_Brain_Fashion.ipynb', 'Camino_Del_Guerrero_No_Numpy.ipynb', 'Camino_Del_Guerrero_Numpy.ipynb', 'Conv_MaxPooling2D.ipynb', 'First_Neural_Network_OR_Gate.py', 'Hello.py', 'Neural_Network_AND_Gate.py', 'TrampaMortalFashionMNIST.ipynb', 'zapato_mike.png']
Modelos encontrados: ['.\\.venv\\Lib\\site-packages\\h5py\\tests\\data_files\\vlen_string_dset.h5', '.\\.venv\\Lib\\site-packages\\h5py\\tests\\data_files\\vlen_string_dset_utc.h5', '.\\.venv\\Lib\\site-packages\\h5py\\tests\\data_files\\vlen_string_s390x.h5']


ValueError: No model config found in the file at .\.venv\Lib\site-packages\h5py\tests\data_files\vlen_string_dset.h5.

In [5]:
predecir_ropa('zapato_mike.png')

# save example (run where `model` is available)
model.save('mi_modelo.keras')

ValueError: File not found: filepath=mi_modelo.keras. Please ensure the file is an accessible `.keras` zip file.